In [15]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from src.env.DunderBotEnv import DunderBotEnv
import src.run as run
import src.load as load

In [17]:
from src.util.config import get_config
config = get_config()

# TODO:

- setup train/test functionality (separate reset functions? Multiple time periods for testing?)
- figure out how to normalize values properly
    * do we have to account for prediction as well? That is, can/should we predict using values outside of 0-1 range?
- normalize the action space?
- understand the rendering during learning
- Change policy network (need to understand terminology surronding it, e.g. envoronment, n_steps etc). See also discussion here (and elsewhere among Issues: https://github.com/hill-a/stable-baselines/issues/759)
- Check/Enable/Add exploration noise (for PPO2, use the entropy flag), use https://stable-baselines.readthedocs.io/en/master/common/schedules.html
- lock down the seeds (and decide in config from a few specific ones)
- setup performance output, what we need to make data driven decision
    *e.g. https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/1_getting_started.ipynb#scrollTo=63M8mSKR-6Zt
- Add benchmarks
- Add checks and tests
- Find as long time series of BTCUSD hourly data as possible, quite shortright now
- Add finer temporal resolution in data (generate/download multiple granularity)
- Check float number type
- self.stationarize_rewards?
- Enable saveing/loading trained models


# DONE: 
- add config
- modularize rewards
- own actions
- add more detailed actions for buy and sell amounts (still Discrete, I believe)
- wrap running code in function in main.py (to enable more dynamic data loading and train/test)
- own observations
- remove the bias when getting the observations (suggestion in the repo issues)
- download more interesting data and manually preprocess
- load interesting data, specified in config
- understand the time steps in learn and predict
- setup basic monitoring of training using TensorBoard
- Visualize basic performance
- Add support for fractions of shares
- Understand what should belong in what timestep (also decide on which price to plot as a result)
- add distributions of actions as output during prediction
- Refactor `_take_action` (might be something off in some calculations, also update terminology)
- Add commission and slippage
- Refactor current reward and add some new ones
- Add smallest amount of bitcoin we can own, as well as trading precision
- Add portfolio features
- Add reward output

# Run

In [18]:
%%time
df = load.load_OHLC_data()
env = run.preprocess(df=df)
model = run.train(env=env, timesteps=1000)

20628 6000 26628
Resetting to timesteps: start 9, end 20628.
--------------------------------------
| approxkl           | 5.6397403e-06 |
| clipfrac           | 0.0           |
| explained_variance | 0.111         |
| fps                | 112           |
| n_updates          | 1             |
| policy_entropy     | 2.197219      |
| policy_loss        | -0.0008178451 |
| serial_timesteps   | 128           |
| time_elapsed       | 0.00021       |
| total_timesteps    | 128           |
| value_loss         | 2.3662735e-05 |
--------------------------------------
CPU times: user 9.04 s, sys: 574 ms, total: 9.61 s
Wall time: 8.88 s


In [19]:
%%time
timesteps = config.train_test.test_timesteps
run.predict(df=df, model=model, timesteps=timesteps, rendermode='human')

20628 6000 26628
Resetting to timesteps: start 20637, end 26628.


ValueError: Length of passed values is 6001, index implies 0

In [6]:
import pickle
with open('all_dict_pred.pickle', 'rb') as f:
    x = pickle.load(f)
x

{'current_step': 20050,
 'net_worths': [10000.0,
  10000.0,
  10018.25,
  10015.38,
  10013.39,
  10006.04,
  10018.63,
  10023.27,
  10019.52,
  10019.91,
  10019.26,
  10016.1,
  10011.38,
  10013.64,
  10014.76,
  10014.16,
  10012.56,
  10026.73,
  10058.07,
  10072.02,
  10069.27,
  10067.56,
  10078.51,
  10071.75,
  10069.63,
  10088.78,
  9886.7,
  9876.85,
  9899.92,
  9894.92,
  9888.64,
  9902.74,
  9906.12,
  9898.2,
  9895.64,
  9918.94,
  9922.17,
  9910.99,
  9913.99,
  9912.68,
  9917.1,
  9899.99,
  9862.58,
  9879.17,
  9870.11,
  9864.2,
  9895.84,
  9895.59,
  9900.92,
  9888.22,
  9890.41],
 'trades': [{'step': 20000,
   'amount': 0.6012506,
   'total': 5000.0,
   'type': 'buy',
   'action_amount': 0.5},
  {'step': 20001,
   'amount': 0.15031265,
   'total': 1254.56,
   'type': 'sell',
   'action_amount': 0.25},
  {'step': 20002,
   'amount': 0.09018759,
   'total': 752.16,
   'type': 'sell',
   'action_amount': 0.2},
  {'step': 20003,
   'amount': 0.07215007,
   '

In [8]:
x.keys()

dict_keys(['current_step', 'net_worths', 'trades', 'account_history', 'rewards'])

# Traintest TODO:
- Set done criterion for end_step: for train, it should start over. FOr test it should not, right?
- plot length should be decided by start step and current step (in case we don't run the whole test period)
- Make sure the overlap is 0 (I think it is 1 right now)